In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import when
from pyspark.sql.functions import when, col, countDistinct
from pyspark.sql.functions import posexplode
from pyspark.sql.functions import posexplode_outer
from pyspark.sql.functions import expr
from pyspark.sql.functions import pandas_udf
from pyspark.sql.functions import PandasUDFType
import pandas as pd

Welcome to the Glue Interactive Sessions Kernel
For more information on available magic commands, please type %help in any new cell.

Please view our Getting Started page to access the most up-to-date information on the Interactive Sessions kernel: https://docs.aws.amazon.com/glue/latest/dg/interactive-sessions.html
Installed kernel version: 0.38.1 
Authenticating with environment variables and user-defined glue_role_arn: arn:aws:iam::142401413602:role/service-role/AmazonSageMaker-ExecutionRole-20220518T150261
Trying to create a Glue session for the kernel.
Worker Type: G.1X
Number of Workers: 5
Session ID: fa2a0605-e312-48a4-9754-4efa6a4c3164
Job Type: glueetl
Applying the following default arguments:
--glue_kernel_version 0.38.1
--enable-glue-datacatalog true
Waiting for session fa2a0605-e312-48a4-9754-4efa6a4c3164 to get into ready status...
Session fa2a0605-e312-48a4-9754-4efa6a4c3164 has been created.



In [2]:
spark = SparkSession.builder.appName("example").getOrCreate()

In [3]:
df = spark.sql(
"""

select 
 cd_cpf
, id_pessoa
, ds_classif_behavior
, ds_seg_consumo
, ds_perfil_research
, ds_persona
from customer_analytics_zone.ca_analitico_basao_crm

"""
)


In [4]:
df.show(5)

+-----------+--------------------+-------------------+----------------+--------------------+--------------------+
|     cd_cpf|           id_pessoa|ds_classif_behavior|  ds_seg_consumo|  ds_perfil_research|          ds_persona|
+-----------+--------------------+-------------------+----------------+--------------------+--------------------+
|77835301768|40caa0ecde15593dd...|                  4|SUPERMERCADO - 6|       Multiplicador|5 - Mulher, Casad...|
|06757552396|a61b721f5327d3562...|               null|            null|                null|0 - Mulher, Casad...|
|02005519570|a9cc792c8f5d6364e...|                 16|SUPERMERCADO - 5|Multiplicador / B...|16 - Homem, 25 a ...|
|04098389185|3df35c0c5aebbf855...|                 16|            null|       Multiplicador|24 - Mulher, 25 a...|
|04999545306|e267ae1724569dc3c...|                  6|SUPERMERCADO - 2|Multiplicador / A...|15 - Mulher, 25 a...|
+-----------+--------------------+-------------------+----------------+-----------------

In [5]:
df.count()

5543924


In [6]:
unpivotExpr = """
stack(4
, 'ds_classif_behavior', ds_classif_behavior
, 'ds_seg_consumo', ds_seg_consumo
, 'ds_perfil_research', ds_perfil_research
, 'ds_persona', ds_persona) 
as (Modelos,Score)
"""

df_formato = df.select("id_pessoa", expr(unpivotExpr)) \
.withColumnRenamed('Modelos', 'id').withColumnRenamed('Score', 'score')\
.withColumn('description', \
when(col("id") == "ds_classif_behavior", "Behavior").otherwise(\
when(col("id") == "ds_seg_consumo", "Segm.MCC").otherwise(\
when(col("id") == "ds_perfil_research", "Perfis").otherwise(\
when(col("id") == "ds_persona", "Persona").otherwise("")))))



df_formato.show()

+--------------------+-------------------+--------------------+-----------+
|           id_pessoa|                 id|               score|description|
+--------------------+-------------------+--------------------+-----------+
|40caa0ecde15593dd...|ds_classif_behavior|                   4|   Behavior|
|40caa0ecde15593dd...|     ds_seg_consumo|    SUPERMERCADO - 6|   Segm.MCC|
|40caa0ecde15593dd...| ds_perfil_research|       Multiplicador|     Perfis|
|40caa0ecde15593dd...|         ds_persona|5 - Mulher, Casad...|    Persona|
|a61b721f5327d3562...|ds_classif_behavior|                null|   Behavior|
|a61b721f5327d3562...|     ds_seg_consumo|                null|   Segm.MCC|
|a61b721f5327d3562...| ds_perfil_research|                null|     Perfis|
|a61b721f5327d3562...|         ds_persona|0 - Mulher, Casad...|    Persona|
|a9cc792c8f5d6364e...|ds_classif_behavior|                  16|   Behavior|
|a9cc792c8f5d6364e...|     ds_seg_consumo|    SUPERMERCADO - 5|   Segm.MCC|
|a9cc792c8f5

In [10]:
df_formato.coalesce(1).write.csv("s3a://data-workspace-will-prod/flavia-costa/base_modelos2/base_modelos.csv")


Stopping session: fa2a0605-e312-48a4-9754-4efa6a4c3164
